In [ ]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
from cycler import cycler

In [ ]:
df = pd.read_excel('Iso.xlsx', sheet_name='Previous worksheet').set_index('Variable')
df.columns = df.loc['R1 FeedTemperature', :]


In [ ]:
visualization_targets = ['R1 Results C4+ yield (wt) [%]', 'R1 Results C5+ yield (wt) [%]', 
                         'R1 Results PIN', 'R1 Results Product C4+ RON', 'iC4/C4 [%]', 'iC5/C5 [%]', '2MC5/C6 [%]', '3MC5/C6 [%]',
                         '22MB/C6 [%]', '23MB/C6 [%]', 'iC7/C7 [%]']

temps = df.columns[2:]

f, axes = plt.subplots(6, 2, figsize=(20, 20), sharex=True)
axes = axes.flatten()

for i, vt in enumerate(visualization_targets):
    ax = axes[i]
    ax.plot(temps, df.loc[vt, temps], 'o-', markersize=15, linewidth=5)
    ax.set_title(vt, fontsize=18)
    ax.tick_params(axis='x', labelsize=16)
    ax.tick_params(axis='y', labelsize=16)
    ax.grid('on')

In [ ]:
visualization_targets = ['R2 Results C4+ yield (wt) [%]', 
                         'R2 Resutls C5+ yield (wt) [%]', 
                         'R2 Results PIN', 
                         'R2 Results Product C4+ RON', 
                         'R2 Results iC4/C4 [%]',
                         'R2 Results iC5/C5 [%]',
                         'R2 Results 2MC5/C6 [%]',
                         'R2 Resutls 3MC5/C6 [%]',
                         'R2 Resutls 22MB/C6 [%]',
                         'R2 Results 23MB/C6 [%]',
                         'R2 Results iC7/C7 [%]'
                        ]

temps = df.columns[2:]

f, axes = plt.subplots(6, 2, figsize=(20, 20), sharex=True)
axes = axes.flatten()

for i, vt in enumerate(visualization_targets):
    ax = axes[i]
    ax.plot(temps, df.loc[vt, temps], 'o-', markersize=15, linewidth=5)
    ax.set_title(vt, fontsize=18)
    ax.tick_params(axis='x', labelsize=16)
    ax.tick_params(axis='y', labelsize=16)
    ax.grid('on')

# Product Proportions

In [ ]:
df = pd.read_excel('Iso.xlsx', sheet_name='Cleaned Data').set_index('Variable')
temps = np.array(df.loc['R1 FeedTemperature', df.columns[1::2]]).astype(np.float)
products = [
    "R2 Product Hydrogen",
    "R2 Product n-Butane",
    "R2 Product i-Butane",
    "R2 Product n-Pentane",
    "R2 Product i-Pentane",
    "R2 Product n-Hexane",
    "R2 Product 2-Mpentane",
    "R2 Product 3-Mpentane",
    "R2 Product 22-Mbutane",
    "R2 Product 23-Mbutane",
    "R2 Product n-Heptane",
    "R2 Product 2-Mhexane",
    "R2 Product Cyclopentane",
    "R2 Product Mcyclopentan",
    "R2 Product Cyclohexane",
    "R2 Product Benzene",
    "R2 Product Ethane",
    "R2 Product Propane",
    "R2 Product Methane"
]
product_colors = [c['color'] for c in  cycler(color=plt.cm.tab20.colors)]
pc = list(zip(products, product_colors))

f, ax = plt.subplots(1, 1, figsize=(20, 10))
for product, color in pc:
    ax.plot(temps, df.loc[product, df.columns[2::2]], linewidth=5, linestyle='-', marker='o', markersize=15, color=color, label=product)
ax.tick_params(axis='x', labelsize=22)
ax.tick_params(axis='y', labelsize=22)
ax.set_xlabel('Temperature (C)', fontsize=28, fontweight='bold')
ax.set_ylabel('Proportion', fontsize=28, fontweight='bold')
ax.grid(True)
ax.legend(loc='upper center', bbox_to_anchor=(0.5, -0.08), fancybox=False, shadow=False, frameon=False, ncol=3, fontsize=24)
ax.set_title('R2 Product Proportions', fontweight='bold', fontsize=32)

In [ ]:
# pick out products where proportion is always less than 0.1
small_prods = [p for p in pc if np.max(df.loc[p[0], df.columns[2::2]]) < 0.1]
f, ax = plt.subplots(1, 1, figsize=(20, 10))

for product, color in small_prods:
    ax.plot(temps, df.loc[product, df.columns[2::2]], linewidth=5, linestyle='-', marker='o', markersize=15, color=color, label=product)
    ax.tick_params(axis='x', labelsize=22)
    ax.tick_params(axis='y', labelsize=22)
    ax.set_xlabel('Temperature (C)', fontsize=28, fontweight='bold')
    ax.set_ylabel('Proportion', fontsize=28, fontweight='bold')
    ax.grid(True)
    ax.legend(loc='upper center', bbox_to_anchor=(0.5, -0.08), fancybox=False, shadow=False, frameon=False, ncol=3, fontsize=24)
ax.set_title('Products whose Proportion < 0.1', fontweight='bold', fontsize=32)